In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

Reading package lists... Done
Building dependency tree       
Reading state information... Done
coinor-cbc is already the newest version (2.9.9+repack1-1).
coinor-clp is already the newest version (1.16.11+repack1-1).
glpk-utils is already the newest version (4.65-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
# Import PuLP modeler functions
from pulp import *

Zadanie przedstawia 3 fabryki, które produkują okna drewniane oraz aluminiowe. Okna drewniane są droższe, gdyż kosztują 5000, natomiast aluminiowe tańsze o 2 tysiące. Każda z nich ma określoną liczbę godzin dostępności w tygodniu oraz określony jest parametr czasu produkcji, każdego z dwóch rodzaji dla każdej fabryki, lecz nie każda produkuje wszystkie. Zestawienie danych w tabeli:

||Czas produkcji profilu aluminiowego|Czas produkcji profilu drewnianego|ilość godzin dostępna w tygodniu|
|--|--|--|--|
|Fabryka 1|1|-|4|
|Fabryka 2|-|2|12|
|Fabryka 3|3|2|18|

Zysk z produkcji:
* profile aluminiowe 3000
* profile drewniane  5000


Zadanie 1: Znaleźć maksymalny zysk tygodniowy przy podanej wydajności każdej fabryki, gdzie ograniczeniem jest  czas. Liczba partii nie została określona. Jest pełna dowolność.



In [ ]:
prob = LpProblem("Najlepsza produkcja HURT", LpMaximize)

profile_alu = LpVariable("Profile aluminiowe", 0) # juz tutaj podajemy ograniczenie ze sa nieujemne
profile_drew = LpVariable("Profile drewniane", 0)

produkcja_f1 = LpVariable("produkcja fabryki 1", 0)
produkcja_f2 = LpVariable("produkcja fabryki 2", 0)
produkcja_f3 = LpVariable("produkcja fabryki 3", 0)

prob += 3000 * profile_alu + 5000 * profile_drew 
prob += 1 * profile_alu == produkcja_f1,  "ilość partii wyprodukowanej przez F1"
prob += 2 * profile_drew == produkcja_f2, "ilość partii wyprodukowanej przez F2"
prob += 3 * profile_alu + 2 * profile_drew  == produkcja_f3, "ilość partii wyprodukowanej przez F3"
prob += produkcja_f1 <= 4,  "Maksymalny czas produkcji fabryki 1"
prob += produkcja_f2 <= 12, "Maksymalny czas produkcji fabryki 2"
prob += produkcja_f3 <= 18, "Maksymalny czas produkcji fabryki 3"

prob.solve()
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Zysk wyniesie {} zl ".format(value(prob.objective)))

Status: Optimal
Profile_aluminiowe = 2.0
Profile_drewniane = 6.0
produkcja_fabryki_1 = 2.0
produkcja_fabryki_2 = 12.0
produkcja_fabryki_3 = 18.0
Zysk wyniesie 36000.0 zl 


/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Wynik funkcji celu nie jest satysfakcjonujący, gdyż nie wykorzystał w pełni możliwości fabryki 1. Z tego względu można eykorzystać panel interaktywny do korygowania ilości partii wykorzystując suwaki.

In [ ]:
?? interact

Object `interact` not found.


In [ ]:
from pulp import *
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, FloatSlider, IntSlider
import ipywidgets as widgets
import pandas as pd

style = {'description_width': 'initial'}


produkcja_f1_slider = IntSlider(min=0,max = 20, value= 2, description="produkcja_f1_slider", style= style )
produkcja_f2_slider = IntSlider(min=0,max = 20,value= 12, description="produkcja_f2_slider", style= style )
produkcja_f3_slider = IntSlider(min=0,max = 20, value= 18, description="produkcja_f3_slider", style= style )
zysk_alu_slider = IntSlider(min=0,max = 10000, value= 3000, description="Zysk_z_aluminiowych_okien", style= style )
zysk_drew_slider = IntSlider(min=0,max = 10000, value= 5000, description="Zysk_z_drewnianych_okien", style= style )



def hurt_produkcja(
    produkcja_f1 = 4,
    produkcja_f2 = 12,
    produkcja_f3 = 18,      
    zysk_alu = 3000,
    zysk_drew = 5000):
  
  prob = LpProblem("Najlepsza produkcja HURT",LpMaximize)

  profile_alu = LpVariable("Profile aluminiowe", 0) # juz tutaj podajemy ograniczenie ze sa nieujemne
  profile_drew = LpVariable("Profile drewniane", 0)

  prob += zysk_alu * profile_alu + zysk_drew * profile_drew, "zysk z produkcji" 
  prob += 1 * profile_alu <= produkcja_f1,  "ilość partii wyprodukowanej przez F1"
  prob += 2 * profile_drew <= produkcja_f2, "ilość partii wyprodukowanej przez F2"
  prob += 3 * profile_alu + 2 * profile_drew  <= produkcja_f3, "ilość partii wyprodukowanej przez F3"

  
  prob.solve()
  print("Status:", LpStatus[prob.status])
    # The status of the solution is printed to the screen
   
    # Each of the variables is printed with it's resolved optimum value
  for v in prob.variables():
      print(v.name, "=", v.varValue)
  print("Zysk wyniesie {} zł  ".format( value(prob.objective)))
    
  shadows = [{'name':name, 'shadow price': c.pi, "slack": c.slack} for name, c in prob.constraints.items()]
  print("*"*10+"Shadow Prices"+"*"*10)
  print(pd.DataFrame(shadows))
    
    
interact(hurt_produkcja,
    zysk_alu= zysk_alu_slider,
    zysk_drew = zysk_drew_slider,
    produkcja_f1 = produkcja_f1_slider,
    produkcja_f2 = produkcja_f2_slider,
    produkcja_f3 = produkcja_f3_slider)


interactive(children=(IntSlider(value=2, description='produkcja_f1_slider', max=20, style=SliderStyle(descript…

<function __main__.hurt_produkcja>